In [1]:
#import squidpy as sq
import spatialdata as sd
import spatialdata_io as sio
from spatialdata import read_zarr
import pandas as pd
#import spatialdata_plot

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
xenium_path = "./data/Xenium_V1_Human_Kidney_FFPE_Protein_updated_outs/"

sdata = sio.xenium(xenium_path, gex_only=False, morphology_focus=False, cells_boundaries=False, nucleus_boundaries=False, cells_labels=False, nucleus_labels=False, cells_as_circles=True)

print(sdata)


C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\anndata\_core\anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\thoma\AppData\Local\Programs\Python\Python312\Lib\functools.py:907: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


SpatialData object
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── Shapes
│     └── 'cell_circles': GeoDataFrame shape: (465534, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (465534, 543)
with coordinate systems:
    ▸ 'global', with elements:
        transcripts (Points), cell_circles (Shapes)


In [3]:
print(sdata.tables)
print('------------')
print(sdata.tables['table'].var)


{'table': AnnData object with n_obs × n_vars = 465534 × 543
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'}
------------
                   gene_ids       feature_types   genome
ABCC11      ENSG00000121270     Gene Expression  Unknown
ACE2        ENSG00000130234     Gene Expression  Unknown
ACKR1       ENSG00000213088     Gene Expression  Unknown
ACTA2       ENSG00000107796     Gene Expression  Unknown
ACTG2       ENSG00000163017     Gene Expression  Unknown
...                     ...                 ...      ...
PanCK             TXP000017  Protein Expression  Unknown
Vimentin          TXP000022  Protein Expression  Unknown
alphaSMA          TXP000001  Prote

In [4]:
# Récupère l'AnnData
adata = sdata.tables["table"]

# Crée un masque pour les protéines
prot_mask = adata.var["feature_types"] == "Protein Expression"

# Crée un masque pour les gènes
gene_mask = adata.var["feature_types"] == "Gene Expression"

# Pour la cellule 0
i = 1

# Extraire les ARN pour la cellule 0
rna_subset = adata[i, gene_mask]
rna_values = rna_subset.X.toarray().flatten() if hasattr(rna_subset.X, "toarray") else rna_subset.X.flatten()

# Extraire les protéines pour la cellule 0
prot_subset = adata[i, prot_mask]
prot_values = prot_subset.X.toarray().flatten() if hasattr(prot_subset.X, "toarray") else prot_subset.X.flatten()

# Afficher les noms de protéines + valeurs
prot_names = adata.var_names[prot_mask]
for name, val in zip(prot_names, prot_values):
    print(f"{name}: {val}")

PD-1: 0.0
VISTA: 9.899999618530273
PD-L1: 0.0
LAG-3: 0.0
CD16: 7.099999904632568
GranzymeB: 0.0
CD163: 0.0
CD4: 2.9000000953674316
CD20: 0.0
CD8A: 0.0
CD3E: 1.0
CD138: 0.0
HLA-DR: 28.5
CD11c: 0.0
CD68: 21.899999618530273
CD45RA: 0.0
PCNA: 2.200000047683716
CD45RO: 0.0
Ki-67: 0.0
Beta-catenin: 0.0
CD31: 0.699999988079071
PTEN: 8.300000190734863
PanCK: 0.0
Vimentin: 8.199999809265137
alphaSMA: 0.0
CD45: 23.899999618530273
E-Cadherin: 1.7999999523162842


In [5]:
cell_positions = sdata.shapes["cell_circles"]  # ou vérifiez le nom exact avec print(sdata.shapes)
cell_0_position = cell_positions.iloc[i]

# Afficher les coordonnées
print(f"Position de la cellule {i}:")
print(f"x: {cell_0_position.geometry.x}")
print(f"y: {cell_0_position.geometry.y}")
print(f"radius: {cell_0_position}")

Position de la cellule 1:
x: 409.8421325683594
y: 2682.607666015625
radius: geometry    POINT (409.8421325683594 2682.607666015625)
radius                                         5.202465
Name: aaaadpem-1, dtype: object


In [6]:
# print position for adata obsm
print(adata.obsm["spatial"][i])


[ 409.84213257 2682.60766602]


In [7]:
print(adata.var["feature_types"].value_counts())

feature_types
Gene Expression              405
Negative Control Codeword     41
Unassigned Codeword           35
Protein Expression            27
Negative Control Probe        20
Deprecated Codeword           15
Name: count, dtype: int64


In [8]:
print(adata.var_names)


Index(['ABCC11', 'ACE2', 'ACKR1', 'ACTA2', 'ACTG2', 'ADAM28', 'ADAMTS1',
       'ADGRE1', 'ADGRL4', 'ADH1C',
       ...
       'CD45RO', 'Ki-67', 'Beta-catenin', 'CD31', 'PTEN', 'PanCK', 'Vimentin',
       'alphaSMA', 'CD45', 'E-Cadherin'],
      dtype='object', length=543)


In [9]:
# # Pipeline GNN pour prédiction de coordonnées spatiales
#
# Ce notebook implémente un Graph Neural Network (GAT) qui prédit les coordonnées
# spatiales des cellules basé uniquement sur leurs profils d'expression d'ARN et de protéines.
# Le graphe K-NN est construit sur la similarité d'expression (pas les coordonnées).
#
# ## ⚙️ Deux approches disponibles :
#
# ### Approche 1 : Graphe Global (défaut)
# - **Un seul grand graphe** avec toutes les cellules
# - Chaque cellule connectée à ses k voisins les plus proches
# - Le GNN traite tout le graphe en une fois
# - **Plus rapide** mais moins flexible
# - Chaque cellule voit indirectement toutes les autres via les couches GNN
#
# ### Approche 2 : Sous-graphes Locaux (recommandé pour votre question)
# - **Un sous-graphe par cellule** : 1 cellule centrale + 29 voisins = 30 nœuds
# - Le GNN prédit UNIQUEMENT la position de la cellule centrale
# - Traitement par batches de sous-graphes
# - **Plus conforme à votre description** : chaque point d'entraînement = 1 sous-graphe
# - Isolement complet : chaque prédiction utilise uniquement son voisinage local


In [10]:
# CHOISIR L'APPROCHE ICI
USE_SUBGRAPHS = True  # True = sous-graphes locaux, False = graphe global

print(f"{'='*60}")
if USE_SUBGRAPHS:
    print("🎯 Approche: SOUS-GRAPHES LOCAUX")
    print("   • Chaque point = 1 cellule centrale + 29 voisins")
    print("   • Prédiction uniquement de la cellule centrale")
    print("   • Traitement par batches")
else:
    print("🎯 Approche: GRAPHE GLOBAL")
    print("   • Un seul grand graphe avec toutes les cellules")
    print("   • Chaque cellule prédit sa propre position")
    print("   • Traitement full-batch")
print(f"{'='*60}\n")


🎯 Approche: SOUS-GRAPHES LOCAUX
   • Chaque point = 1 cellule centrale + 29 voisins
   • Prédiction uniquement de la cellule centrale
   • Traitement par batches



In [11]:
# Imports pour le pipeline GNN
import torch
from scipy.sparse import issparse

if USE_SUBGRAPHS:
    from data_preprocessing import preprocess_adata
    from data_preprocessing_subgraph import build_local_subgraphs, create_subgraph_splits
    from train_subgraph import SubgraphTrainer as Trainer
else:
    from data_preprocessing import preprocess_adata, build_knn_graph, create_train_val_test_masks
    from train import Trainer

from model import create_model
from evaluate import (evaluate_predictions, plot_training_history,
                      plot_predictions_vs_true, plot_spatial_predictions,
                      plot_error_distribution, analyze_extreme_errors)


C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# ## 1. Préparation et normalisation des données


In [13]:
# Prétraiter les données (filtrer et normaliser)
adata_processed = preprocess_adata(adata, normalize_genes=True, normalize_proteins=True)


C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\anndata\_core\anndata.py:1798: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Nombre de cellules: 465534
Nombre de gènes: 405
Nombre de protéines: 27
Total features: 432


C:\Users\thoma\OneDrive\Bureau\THOMAS\Programmes_Informatiques\Stat ML Project\.venv\Lib\site-packages\legacy_api_wrap\__init__.py:88: UserWarning: Some cells have zero counts
  return fn(*args_all, **kw)


✓ Gènes normalisés (normalize_total + log1p)
✓ Protéines normalisées (z-score)


In [14]:
# Extraire les features et coordonnées spatiales
if issparse(adata_processed.X):
    features = adata_processed.X.toarray()
else:
    features = adata_processed.X

spatial_coords = adata_processed.obsm["spatial"]

print(f"Shape des features: {features.shape}")
print(f"Shape des coordonnées: {spatial_coords.shape}")


Shape des features: (465534, 432)
Shape des coordonnées: (465534, 2)


In [15]:
# ## 2. Construction du graphe K-NN basé sur similarité d'expression


In [ ]:
if USE_SUBGRAPHS:
    # Approche sous-graphes locaux
    print("Construction des sous-graphes locaux...")
    subgraphs_list, coords_scaler = build_local_subgraphs(
        features=features,
        spatial_coords=spatial_coords,
        k=29,
        metric='cosine'
    )

    # Créer les splits d'indices
    train_indices, val_indices, test_indices = create_subgraph_splits(
        n_subgraphs=len(subgraphs_list),
        train_ratio=0.7,
        val_ratio=0.15,
        test_ratio=0.15,
        seed=42
    )

    # Pour compatibilité avec le reste du code
    data = subgraphs_list[0]  # Juste pour afficher les infos
    print(f"\nExemple de sous-graphe:")
    print(f"  • Nœuds: {data.x.shape[0]} (1 centrale + {data.x.shape[0]-1} voisins)")
    print(f"  • Features par nœud: {data.x.shape[1]}")
    print(f"  • Arêtes: {data.edge_index.shape[1]}")
    print(f"  • Cible: position de la cellule centrale uniquement")

else:
    # Approche graphe global
    print("Construction du graphe global...")
    data, coords_scaler = build_knn_graph(
        features=features,
        spatial_coords=spatial_coords,
        k=29,
        metric='cosine'
    )

    # Créer les masques train/val/test
    train_mask, val_mask, test_mask = create_train_val_test_masks(
        n_nodes=data.x.shape[0],
        train_ratio=0.7,
        val_ratio=0.15,
        test_ratio=0.15,
        seed=42
    )


Construction des sous-graphes locaux...

Construction des sous-graphes locaux (k=29, metric=cosine)...


In [ ]:
# ## 3. Création du modèle GAT


In [ ]:
# Créer le modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device utilisé: {device}")

if USE_SUBGRAPHS:
    in_channels = subgraphs_list[0].x.shape[1]
else:
    in_channels = data.x.shape[1]

model = create_model(
    in_channels=in_channels,  # 432 features (405 gènes + 27 protéines)
    model_type='base',  # ou 'large' pour un modèle plus profond
    hidden_channels=128,
    heads=4,
    dropout=0.4
)


In [ ]:
# ## 4. Entraînement du modèle


In [ ]:
# Créer le trainer
if USE_SUBGRAPHS:
    trainer = Trainer(
        model=model,
        subgraphs_list=subgraphs_list,
        train_indices=train_indices,
        val_indices=val_indices,
        test_indices=test_indices,
        batch_size=32,
        lr=0.001,
        weight_decay=5e-4,
        device=device
    )
else:
    trainer = Trainer(
        model=model,
        data=data,
        train_mask=train_mask,
        val_mask=val_mask,
        test_mask=test_mask,
        lr=0.001,
        weight_decay=5e-4,
        device=device
    )


In [ ]:
# Entraîner le modèle
best_model_state = trainer.train(
    epochs=200,
    early_stopping_patience=20,
    verbose=True
)


In [ ]:
# Visualiser l'historique d'entraînement
history = trainer.get_history()
plot_training_history(history, save_path='results/training_history.png')


In [ ]:
# ## 5. Évaluation sur l'ensemble de test


In [ ]:
# Prédire sur l'ensemble de test
if USE_SUBGRAPHS:
    # Pour les sous-graphes, utiliser la méthode spécifique
    y_pred_test, y_true_test = trainer.predict_all(
        trainer.test_loader,
        denormalize=True,
        coords_scaler=coords_scaler
    )
else:
    # Pour le graphe global
    predictions_normalized = trainer.predict(denormalize=False)
    predictions = trainer.predict(denormalize=True, coords_scaler=coords_scaler)

    # Coordonnées réelles
    y_true = data.pos_original.cpu().numpy()

    # Filtrer pour l'ensemble de test
    y_true_test = y_true[test_mask.cpu().numpy()]
    y_pred_test = predictions[test_mask.cpu().numpy()]


In [ ]:
# Calculer les métriques
metrics, euclidean_distances = evaluate_predictions(
    y_true_test,
    y_pred_test,
    set_name='Test'
)


In [ ]:
# Visualiser prédictions vs réalité
plot_predictions_vs_true(y_true_test, y_pred_test,
                        save_path='results/predictions_vs_true.png')


In [ ]:
# Visualiser les positions spatiales
plot_spatial_predictions(y_true_test, y_pred_test, euclidean_distances,
                        save_path='results/spatial_predictions.png')


In [ ]:
# Distribution des erreurs
plot_error_distribution(euclidean_distances,
                       save_path='results/error_distribution.png')


In [ ]:
# Analyser les erreurs extrêmes
worst_cells, best_cells = analyze_extreme_errors(
    y_true_test,
    y_pred_test,
    euclidean_distances,
    top_n=10
)


In [ ]:
# ## 6. Sauvegarder le modèle


In [ ]:
# Créer le dossier results s'il n'existe pas
import os
os.makedirs('results', exist_ok=True)

# Sauvegarder le modèle
trainer.save_model('results/spatial_gat_model.pt')


In [ ]:
# Sauvegarder les métriques dans un fichier CSV
import pandas as pd
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv('results/test_metrics.csv', index=False)
print("✓ Métriques sauvegardées dans results/test_metrics.csv")
